In [18]:
import pandas as pd
import sys
import numpy as np
sys.path.append('/home/tammy.kojima/Authorship-attribution/')
import feature_extraction as fe
from tqdm import tqdm
import pickle
import os
import ast
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_gpt_com_features.csv')
df.head()

,text,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_random_uppercase,stylistic_repeated_chars,...,complexity_long_sentences,complexity_rare_words,complexity_syntactic_depth,temporal_recent_ratio,temporal_past_ratio,temporal_future_ratio,llm_chatgpt_patterns,llm_structured_output,llm_overly_polite,llm_disclaimer_density
0,"Our rules are the most robust, the president c...",1,0.882353,17,15,5.294118,6.442907,0.411765,0,0,...,1.0,0.764706,1.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,...associating outsiders with danger and extre...,1,1.000000,18,18,6.000000,9.000000,0.277778,0,1,...,0.0,0.777778,1.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,...alarms were raised in the initial stages of...,1,0.937500,32,30,4.968750,4.655273,0.437500,0,1,...,1.0,0.812500,2.857143,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,[The opposition] utilizes every tool to questi...,1,0.727273,22,16,5.409091,8.787190,0.500000,0,0,...,1.0,0.590909,2.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,[The governor's] fixation with health protocol...,1,0.923077,13,12,6.153846,7.207101,0.384615,0,0,...,1.0,0.769231,1.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df = df.drop(columns=["ai_perplexity_score", "ai_repeated_ngrams", "llm_chatgpt_patterns", "llm_structured_output", "llm_overly_polite", "llm_disclaimer_density"])
df.head()

,text,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_random_uppercase,stylistic_repeated_chars,...,with the,work,working,world,would,year,years,you,you are,your
0,"Our rules are the most robust, the president c...",1,0.882353,17,15,5.294118,6.442907,0.411765,0,0,...,0,0,0,0,0,0,0,0,0,0
1,...associating outsiders with danger and extre...,1,1.000000,18,18,6.000000,9.000000,0.277778,0,1,...,0,0,0,0,0,0,0,0,0,0
2,...alarms were raised in the initial stages of...,1,0.937500,32,30,4.968750,4.655273,0.437500,0,1,...,0,0,0,0,0,0,0,0,0,0
3,[The opposition] utilizes every tool to questi...,1,0.727273,22,16,5.409091,8.787190,0.500000,0,0,...,0,0,0,0,0,0,0,0,0,0
4,[The governor's] fixation with health protocol...,1,0.923077,13,12,6.153846,7.207101,0.384615,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_gpt = pd.read_csv('df_pronto/df_gpt.csv')
df_human = pd.read_csv('df_pronto/df_human.csv')
#df_gpt = df_gpt.iloc[:10000]
df_human = df_human.iloc[:100000]
df_test = pd.concat([df_gpt, df_human], ignore_index=True)
df_test

,text,origin,model
0,"Our rules are the most robust, the president c...",AI,chat gpt
1,...associating outsiders with danger and extre...,AI,chat gpt
2,...alarms were raised in the initial stages of...,AI,chat gpt
3,[The opposition] utilizes every tool to questi...,AI,chat gpt
4,[The governor's] fixation with health protocol...,AI,chat gpt
...,...,...,...
199995,modi took revenge you bcos you are advani team...,human,NaN
199996,really doubt any organisation allowed function...,human,NaN
199997,⃣ already possession this capabilities but thi...,human,NaN
199998,although nasa doesnt have space suits for wome...,human,NaN


In [16]:
# 2. Selecionar a coluna de textos
# (substitua 'texto' pelo nome real da coluna com o conteúdo)
textos = df_test["text"].fillna("").tolist()

In [ ]:
# Tokeniza todos os textos primeiro
processed_texts = [fe.preprocess_text(texto) for texto in tqdm(textos, desc="Tokenizando textos")]

Extraindo features estilísticas: 100%|██████████| 200000/200000 [00:09<00:00, 21115.84it/s]


,stylistic_random_uppercase,stylistic_repeated_chars,stylistic_repeated_words,stylistic_exclamation_density,stylistic_question_density,stylistic_ellipsis_count,stylistic_emoji_density,stylistic_emoticon_density,stylistic_capitalization_inconsistency
0,0,0,0,0.0,0.0,0,0.0,0.0,0.000000
1,0,1,0,0.0,0.0,1,0.0,0.0,0.055556
2,0,1,0,0.0,0.0,1,0.0,0.0,0.000000
3,0,0,0,0.0,0.0,0,0.0,0.0,0.000000
4,0,0,0,0.0,0.0,0,0.0,0.0,0.000000


In [ ]:
df_prossessed_texts = pd.DataFrame({'text': processed_texts})
df_prossessed_texts.to_csv('df_pronto/df_tokens_gpt.csv', index=False)

In [ ]:
# Lista para armazenar resultados
lexical_features = []

for texto in tqdm(textos):
    # Tokeniza o texto usando Twokenizer
    words = fe.preprocess_text(texto)  # já retorna tokens em lowercase
    
    # Extrai features léxicas
    features = fe.extract_lexical_features(texto, words)
    lexical_features.append(features)

df_lexical = pd.DataFrame(lexical_features)

100%|██████████| 200000/200000 [37:08<00:00, 89.73it/s]  


In [19]:
df_test = pd.concat([df_test, df_lexical], axis=1)
df_test.head()

,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio
0,"Our rules are the most robust, the president c...",AI,chat gpt,0.882353,17,15,5.294118,6.442907,0.411765
1,...associating outsiders with danger and extre...,AI,chat gpt,1.000000,18,18,6.000000,9.000000,0.277778
2,...alarms were raised in the initial stages of...,AI,chat gpt,0.937500,32,30,4.968750,4.655273,0.437500
3,[The opposition] utilizes every tool to questi...,AI,chat gpt,0.727273,22,16,5.409091,8.787190,0.500000
4,[The governor's] fixation with health protocol...,AI,chat gpt,0.923077,13,12,6.153846,7.207101,0.384615


In [ ]:
stylistic_features = [
    fe.extract_stylistic_features(textos[i], len(processed_texts[i]))
    for i in tqdm(range(len(textos)), desc="Extraindo features estilísticas")
]
df_syntactic = pd.DataFrame(stylistic_features)
df_syntactic.head()

In [25]:
df_test = pd.concat([df_test, df_syntactic], axis=1)
df_test.head()

,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_random_uppercase,stylistic_repeated_chars,stylistic_repeated_words,stylistic_exclamation_density,stylistic_question_density,stylistic_ellipsis_count,stylistic_emoji_density,stylistic_emoticon_density,stylistic_capitalization_inconsistency
0,"Our rules are the most robust, the president c...",AI,chat gpt,0.882353,17,15,5.294118,6.442907,0.411765,0,0,0,0.0,0.0,0,0.0,0.0,0.000000
1,...associating outsiders with danger and extre...,AI,chat gpt,1.000000,18,18,6.000000,9.000000,0.277778,0,1,0,0.0,0.0,1,0.0,0.0,0.055556
2,...alarms were raised in the initial stages of...,AI,chat gpt,0.937500,32,30,4.968750,4.655273,0.437500,0,1,0,0.0,0.0,1,0.0,0.0,0.000000
3,[The opposition] utilizes every tool to questi...,AI,chat gpt,0.727273,22,16,5.409091,8.787190,0.500000,0,0,0,0.0,0.0,0,0.0,0.0,0.000000
4,[The governor's] fixation with health protocol...,AI,chat gpt,0.923077,13,12,6.153846,7.207101,0.384615,0,0,0,0.0,0.0,0,0.0,0.0,0.000000


In [ ]:
# especifico para casos com letra maiuscula 
df_test = pd.read_csv('df_pronto/df_2pronto.csv')
textos = df_test["text"].fillna("").tolist()
processed_texts = [fe.preprocess_text_A(texto) for texto in tqdm(textos, desc="Tokenizando textos")]
stylistic_features = [
    fe.extract_stylistic_featuresA(textos[i], len(processed_texts[i]))
    for i in tqdm(range(len(textos)), desc="Extraindo features estilísticas")
]
df_syntactic = pd.DataFrame(stylistic_features)
df_syntactic.head()

/tmp/ipykernel_2927578/763372016.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('df_pronto/df_2pronto.csv')
Extraindo features estilísticas: 100%|██████████| 200000/200000 [00:03<00:00, 64147.59it/s]


,stylistic_random_uppercase,stylistic_capitalization_inconsistency
0,0,0.000000
1,0,0.047619
2,0,0.000000
3,0,0.000000
4,0,0.000000


In [ ]:
df_test = pd.concat([df_test, df_syntactic], axis=1)

In [14]:
structural_features = [
    fe.extract_structural_features(textos[i], processed_texts[i], len(processed_texts[i]))
    for i in tqdm(range(len(textos)), desc="Extraindo features estruturais")
]
df_structural = pd.DataFrame(structural_features)
df_structural.head()

































































































































































































Extraindo features estruturais: 100%|██████████| 200000/200000 [00:04<00:00, 41548.03it/s]


,structural_has_url,structural_has_mention,structural_has_hashtag,structural_is_retweet,structural_url_density,structural_mention_density,structural_hashtag_density,structural_extra_spaces,structural_temporal_markers
0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0.0,0.0,0.0,0.0,0.0


In [16]:
df_test = pd.concat([df_test, df_structural], axis=1)
df_test.to_csv('df_pronto/df_3pronto.csv', index=False)

In [2]:
df_processed_texts = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_tokens_gpt.csv')
df_test = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_3pronto.csv')
processed_texts = df_processed_texts['processed_text'].apply(ast.literal_eval).tolist()
textos = df_test["text"].fillna("").tolist()

/tmp/ipykernel_745779/1100540465.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_3pronto.csv')


In [3]:
# falta fazer esse 
syntactic_features = [
    fe.extract_syntactic_features(tokens)
    for tokens in tqdm(processed_texts, desc="Extraindo features sintáticas")
]
df_syntactic = pd.DataFrame(syntactic_features)
df_syntactic.head()

Extraindo features sintáticas: 100%|█| 200000/200


,syntactic_pos_tag_entropy,syntactic_pos_bigram_entropy,syntactic_avg_sentence_length,syntactic_subordinating_conj,syntactic_comma_ratio,syntactic_punct_ratio
0,1.936199,2.599302,17.0,0.0,0.0,0.0
1,1.875122,2.639341,18.0,0.0,0.0,0.0
2,1.958491,3.031515,32.0,0.0,0.0,0.0
3,1.648841,2.193785,22.0,0.0,0.0,0.0
4,1.631345,2.253858,13.0,0.0,0.0,0.0


In [5]:
df_test = pd.concat([df_test, df_syntactic], axis=1)
df_test.head()

,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_random_uppercase,...,structural_mention_density,structural_hashtag_density,structural_extra_spaces,structural_temporal_markers,syntactic_pos_tag_entropy,syntactic_pos_bigram_entropy,syntactic_avg_sentence_length,syntactic_subordinating_conj,syntactic_comma_ratio,syntactic_punct_ratio
0,"Our rules are the most robust, the president c...",AI,chat gpt,0.882353,17,15,5.294118,6.442907,0.411765,0,...,0.0,0.0,0.0,0.0,1.936199,2.599302,17.0,0.0,0.0,0.0
1,...associating outsiders with danger and extre...,AI,chat gpt,1.000000,18,18,6.000000,9.000000,0.277778,0,...,0.0,0.0,0.0,0.0,1.875122,2.639341,18.0,0.0,0.0,0.0
2,...alarms were raised in the initial stages of...,AI,chat gpt,0.937500,32,30,4.968750,4.655273,0.437500,0,...,0.0,0.0,0.0,0.0,1.958491,3.031515,32.0,0.0,0.0,0.0
3,[The opposition] utilizes every tool to questi...,AI,chat gpt,0.727273,22,16,5.409091,8.787190,0.500000,0,...,0.0,0.0,0.0,0.0,1.648841,2.193785,22.0,0.0,0.0,0.0
4,[The governor's] fixation with health protocol...,AI,chat gpt,0.923077,13,12,6.153846,7.207101,0.384615,0,...,0.0,0.0,0.0,0.0,1.631345,2.253858,13.0,0.0,0.0,0.0


In [6]:
df_processed_texts = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_tokens_gpt.csv')

processed_texts = df_processed_texts['processed_text'].apply(ast.literal_eval).tolist()
textos = df["text"].fillna("").tolist()

X_new_features = fe.extract_features_batch(textos, processed_texts = processed_texts)

In [8]:
df_features = pd.DataFrame(X_new_features)
df_features.head()

,ai_perplexity_score,ai_repeated_ngrams,ai_topic_shifts,ai_safety_disclaimers,ai_hedging_language,complexity_subordinating_ratio,complexity_long_sentences,complexity_rare_words,complexity_syntactic_depth,temporal_recent_ratio,...,chatgpt_structured_output,chatgpt_overly_polite,chatgpt_disclaimer_density,chatgpt_assistant_patterns,mistral_self_ref,mistral_structured_density,mistral_technical_jargon,mistral_non_english_density,mistral_step_reasoning,mistral_low_ethical_disclaimers
0,0.882353,0.0,1.714286,0,0.000000,0.0,1.0,0.764706,1.500000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000000,0.0,1.625000,0,0.000000,0.0,0.0,0.777778,1.750000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.937500,0.0,1.227273,0,0.000000,0.0,1.0,0.812500,2.857143,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.727273,0.0,1.250000,0,0.000000,0.0,1.0,0.590909,2.166667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.923077,0.0,2.666667,0,0.076923,0.0,1.0,0.769231,1.333333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df.head()

,text,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_random_uppercase,stylistic_repeated_chars,...,with the,work,working,world,would,year,years,you,you are,your
0,"Our rules are the most robust, the president c...",1,0.882353,17,15,5.294118,6.442907,0.411765,0,0,...,0,0,0,0,0,0,0,0,0,0
1,...associating outsiders with danger and extre...,1,1.000000,18,18,6.000000,9.000000,0.277778,0,1,...,0,0,0,0,0,0,0,0,0,0
2,...alarms were raised in the initial stages of...,1,0.937500,32,30,4.968750,4.655273,0.437500,0,1,...,0,0,0,0,0,0,0,0,0,0
3,[The opposition] utilizes every tool to questi...,1,0.727273,22,16,5.409091,8.787190,0.500000,0,0,...,0,0,0,0,0,0,0,0,0,0
4,[The governor's] fixation with health protocol...,1,0.923077,13,12,6.153846,7.207101,0.384615,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df_test = pd.concat([df, df_features], axis=1)
df_test.head()

,text,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_random_uppercase,stylistic_repeated_chars,...,chatgpt_structured_output,chatgpt_overly_polite,chatgpt_disclaimer_density,chatgpt_assistant_patterns,mistral_self_ref,mistral_structured_density,mistral_technical_jargon,mistral_non_english_density,mistral_step_reasoning,mistral_low_ethical_disclaimers
0,"Our rules are the most robust, the president c...",1,0.882353,17,15,5.294118,6.442907,0.411765,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,...associating outsiders with danger and extre...,1,1.000000,18,18,6.000000,9.000000,0.277778,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,...alarms were raised in the initial stages of...,1,0.937500,32,30,4.968750,4.655273,0.437500,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,[The opposition] utilizes every tool to questi...,1,0.727273,22,16,5.409091,8.787190,0.500000,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,[The governor's] fixation with health protocol...,1,0.923077,13,12,6.153846,7.207101,0.384615,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1]:
# Exemplo de uso básico
ngrams_df = fe.extract_ngrams_features(processed_texts, max_features=300)
ngrams_df.head()

,19,about,about the,action,administration,administration is,after,again,against,all,...,with the,work,working,world,would,year,years,you,you are,your
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Resetar os índices de ambos os DataFrames
df_test_reset = df.reset_index(drop=True)
ngrams_df_reset = ngrams_df.reset_index(drop=True)

# Agora concatenar
df_combined = pd.concat([df_test_reset, ngrams_df_reset], axis=1)

In [12]:
df_test_reset = df.reset_index(drop=True)

In [14]:
# Troca IA → 1 e human → 0
df_test_reset['origin'] = df_test_reset['origin'].replace({'AI': 1, 'human': 0})

df_test_reset.head()

,text,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_random_uppercase,stylistic_repeated_chars,...,with the,work,working,world,would,year,years,you,you are,your
0,"Our rules are the most robust, the president c...",1,0.882353,17,15,5.294118,6.442907,0.411765,0,0,...,0,0,0,0,0,0,0,0,0,0
1,...associating outsiders with danger and extre...,1,1.000000,18,18,6.000000,9.000000,0.277778,0,1,...,0,0,0,0,0,0,0,0,0,0
2,...alarms were raised in the initial stages of...,1,0.937500,32,30,4.968750,4.655273,0.437500,0,1,...,0,0,0,0,0,0,0,0,0,0
3,[The opposition] utilizes every tool to questi...,1,0.727273,22,16,5.409091,8.787190,0.500000,0,0,...,0,0,0,0,0,0,0,0,0,0
4,[The governor's] fixation with health protocol...,1,0.923077,13,12,6.153846,7.207101,0.384615,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df_combined = df_combined.apply(lambda x: x.sparse.to_dense() if pd.api.types.is_sparse(x) else x)
df_combined.describe()

/tmp/ipykernel_745779/91803334.py:1: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  df_combined = df_combined.apply(lambda x: x.sparse.to_dense() if pd.api.types.is_sparse(x) else x)


,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_random_uppercase,stylistic_repeated_chars,stylistic_repeated_words,...,with the,work,working,world,would,year,years,you,you are,your
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.500000,0.939904,20.395885,18.869735,5.145131,10.948684,0.421463,0.324975,0.130065,0.014420,...,0.013875,0.027415,0.012815,0.042780,0.023140,0.017950,0.023415,0.168210,0.014765,0.049805
std,0.500001,0.065652,9.664566,8.529665,1.318695,13.854440,0.159344,0.468367,0.336376,0.119215,...,0.117824,0.170422,0.113406,0.209237,0.159545,0.138665,0.161018,0.503057,0.142888,0.244448
min,0.000000,0.076923,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.903226,12.000000,12.000000,4.375000,4.093333,0.333333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.500000,0.950000,20.000000,19.000000,4.944444,6.616071,0.437500,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,28.000000,26.000000,5.608696,12.290000,0.533333,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,107.000000,76.000000,76.000000,2348.913580,1.000000,1.000000,1.000000,1.000000,...,3.000000,4.000000,2.000000,4.000000,5.000000,4.000000,4.000000,13.000000,6.000000,5.000000


In [15]:
df_combined.isna().sum()

NameError: name 'df_combined' is not defined

In [17]:
df_test_reset.to_csv("/home/tammy.kojima/Authorship-attribution/df_pronto/df_gpt_com_features.csv", index=False)

KeyboardInterrupt: 